# # CP 217: Machine Learning for Cyber-Physical Systems 
## Week 3: Linear Regression, Polynomial Regression, Regularization
***
In this workshop, first we'll look at linear regression method. Briefly, this involves learning a linear regression model from a training set of $(\mathbf{x}, y)$ pairs, where $\mathbf{x}$ is a feature vector and $y$ is a real-valued response variable.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import io
import pandas as pd
#import csv

### 1. Review
In this week's lecture, we saw that a linear model can be expressed as:
$$y = w_0 + \sum_{j = 1}^{m} w_j x_j = \mathbf{w} \cdot \mathbf{x} $$
where 

* $y$ is the *target (or output) variable*;
* $\mathbf{x} = [x_1, \ldots, x_m]$ is a vector of *features* or *predictors* (we define $x_0 = 1$); and
* $\mathbf{w} = [w_0, \ldots, w_m]$ are the *weights*.

To fit the model, we will *minimise* the residual sum of squares (RSS) (simple case) which is also an MLE estimate for linear regression:

$$RSS(\mathbf{w}) = \sum_{i=1}^{n}(y_i - \mathbf{w} \cdot \mathbf{x}_i)^2$$

**Note:** For simplicity, we'll consider the case $m = 1$ (i.e. only one feature excluding the intercept).

***Problem Statement***

You are given the air quality data of few major cities in India. This data includes several air quality related variables such as PM2.5, PM10, NOx, CO etc. and Air quality index (AQI). Your task is to learn a regression model on the given data to predict AQI from the input variables.

In [ ]:
#load air quality index data  (https://www.kaggle.com/datasets/rohanrao/air-quality-data-in-india)
data=pd.read_csv('AQI_data.csv')
data

In [ ]:
data.describe()

In [ ]:
# removing probable outliers (
index_names = data[ data['AQI'] > 600 ].index  # you can use z-score to find the indices of outliers in AQI)
data.drop(index_names, inplace = True) #drop index_names from data

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
x_=data.iloc[:,1]  #choosing second col of data 
y_=data.iloc[:,11] #choosing AQI col of data which will act as target

In [ ]:
# basic preprocessing
x_=np.array(x_)
y_=np.array(y_)

x=[]
y=[]
for i in range(len(data)):
    x.append([x_[i]])
    y.append([y_[i]])
x=np.array(x)
y=np.array(y)    

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(x, y, 'rx')
#plt.scatter(x, y, alpha=0.2, color='red') # use this for scatter plot
plt.ylabel("AQI")
plt.xlabel("PM 2.5")
plt.show()

### Minimising Sum of Squared Residuals: Iterative Solution 

Now we are going to fit a line, $y_i=w_0 + w_1x_i$, to the data you've plotted. We are trying to minimize the error function

$$E(w_0, w_1) = \sum_{i=1}^N(y_i-w_0-w_1x_i)^2$$

with respect to $w_0$ and $w_1$. We can start with an initial guess for $w_1$ (why $w_1$?)

In [ ]:
w_1= 10

Then we use the maximum likelihood update, derived in the lecture to find an estimate for the offset, $w_0$,

$$w_0 = \frac{\sum_{i=1}^N(y_i-w_1 x_i)}{N}$$

In [ ]:
w_0 = (y - w_1*x).mean()
w_0

And now we can make an estimate for the slope of the line, using this estimate of *w_0*,

$$w_1 = \frac{\sum_{i=1}^N (y_i - w_0) \times x_i}{\sum_{i=1}^N x_i^2}$$

In [ ]:
w_1 = #.....over to you
w_1

We can have a look at how good our fit is by computing the prediction across the input space. First create a vector of 'test points'

In [ ]:
#range of x-axis of data
ll=20
ul=300
x_test = np.arange(ll, ul)[:, None]
x_test

Now use this vector to compute some test predictions,

In [ ]:
f_test = ..... #write your code here
f_test

In [ ]:
y_hat = w_0 + w_1*x #.....write your code here
y_hat

Now plot those test predictions with a blue line on the same plot as the data,

In [ ]:
plt.plot(x_test, f_test, 'b-')
plt.plot(x, y, 'rx')

Next compute the quality of the fit by evaluating the average sum of squares error of the prediction over the training samples, $E(w_0,w_1)$

$$E(w_0, w_1) = \sum_{i=1}^N(y_i-w_0-w_1x_i)^2$$

In [ ]:
RSS= ((y - w_0 - w_1*x)**2).sum()#.....write your code here 
RSS

The fit isn't very good, we need to iterate between these parameter updates in a loop to improve the fit, we have to do this several times,

In [ ]:
for i in np.arange(10):
    w_1 = ((y - w_0)*x).sum()/(x*x).sum()
    w_0 = (y - w_1*x).sum()/y.shape[0]
    if i % 10 == 0: 
        print ('iterations', i, 'training error E', ((y - w_0 - w_1*x)**2).sum())
(w_0, w_1)

And let's try plotting the result again

In [ ]:
f_test =   w_0 + w_1*x_test #.....write your code here 
plt.plot(x_test, f_test, 'b-')
plt.plot(x, y, 'rx')

Does more than 10 iterations considerably improve fit in this case?

### Analytical  Solution (Normal Equation): Direct Solution with Linear Algebra

In lecture, we saw that it's possible to solve for the optimal weights $\mathbf{w}^\star$ analytically. The solution is (Maximum Likelihood estimate)
$$\mathbf{w}^* = \left[\mathbf{X}^\top \mathbf{X}\right]^{-1} \mathbf{X}^\top \mathbf{y}$$
where
$$\mathbf{X} = \begin{pmatrix} 
        1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_n 
    \end{pmatrix} 
  \quad \text{and} \quad 
  \mathbf{y} = \begin{pmatrix} 
          y_1 \\ y_2 \\ \vdots \\ y_n
      \end{pmatrix}
$$

We construct $\mathbf{X}$ in the code block below, remembering to include the $x_0 = 1$ column for the bias (intercept).

In [ ]:
X = np.hstack((np.ones_like(x), x))
print(X)

To get the solution using Normal eequation, we solve the following system of linear equations:
$$\mathbf{X}^\top\mathbf{X} \mathbf{w}^\star = \mathbf{X}^\top\mathbf{y}$$

This can be done in numpy using the command `np.linalg.solve`. Dot product can be done using `np.dot`. (Try `np.linalg.solve?` or `np.dot?` in a cell to see what inputs they take", for transpose of a matrix `X` you can use `X.T`)

In [ ]:
np.linalg.solve?

In [ ]:
w =  #.....write your code here 
print(w)

Let's examine the quality of fit for these values for the weights $w_0$ and $w_1$. We create a vector of "test" values `x_test` and a function to compute the predictions according to the model.

In [ ]:
x_test = np.arange(ll, ul)[:, None]

def predict(x_test, w0, w1): 
    return  #.....write your code here 

In [ ]:
w0, w1 = w

Now plot the test predictions with a blue line on the same plot as the data.

In [ ]:
def plot_fit(x_test, y_test, x, y): 
    plt.plot(x_test, y_test, 'b-')
    plt.plot(x, y, 'rx')
    plt.ylabel("AQI")
    plt.xlabel("PM 2.5")
    plt.show()


plot_fit(x_test, predict(x_test, w0, w1), x, y)

We'll compute the residual sum of squares $RSS(w_0,w_1)$ on the training set to measure the goodness of fit.

Expanding out the RSS for this simple case (where $\mathbf{w}=[w_0, w_1]$) we have:
$$RSS(w_0, w_1) = \sum_{i=1}^{n}(y_i - w_0 - w_1 x_i)^2$$

In [ ]:
def compute_RSS(x, y, w0, w1): 
    return  #.....write your code here 

print(compute_RSS(x, y, w0, w1))

### Hold Out Validation

The error we computed above is the *training* error. It doesn't assess the model's generalization ability, it only assesses how well it's performing on the given training data.

In [ ]:
import random

x_train = x
y_train = y

indices_hold_out=random.sample(range(0,84), 30)
#print(indices_hold_out)
x_train = np.delete(x, indices_hold_out)[:,None]
y_train = np.delete(y, indices_hold_out)[:,None]

# Create a hold out set
x_hold_out = x[indices_hold_out]
y_hold_out = y[indices_hold_out]



***Home Exercise:*** Now use the training set and hold out set to compute the errors like above.

You might also consider *leave one out* cross validation, where the model is trained *n-1* times with each data point excluded once from training and used for testing, or *k-fold cross validation* in which the data is split into equal size folds and the model is trained *k* times with each fold used once for testing.

## Polynomial Regression


Now we will consider a more complex polynomial function. Where before we had instances of the form,
$$\phi(\mathbf{x}) = [ 1~ x ]$$ 
now we will be using e.g., 
$$\phi(\mathbf{x}) = [ 1 ~x~ x^2~ x^3~ x^4]$$ 
for a quartic model. Each element $w_i$ of the weight vector corresponds to the coefficient of the input year raised to the $i^{th}$ power. We will consider a range of polynomial models of different orders. 

To implement this we will use *basis functions* which provide a neat way of representing our data instances such that we can still use all the linear models to achieve learn a non-linear model. 

### Data Preparation

The first thing we'll do is plot the training error for the polynomial fit. To do this let's set up some parameters.

In [ ]:
num_data = x.shape[0]
num_pred_data = 30 # how many points to use for plotting predictions
x_pred = np.linspace(ll,ul, num_pred_data)[:, None] # input locations for predictions
order = 4 # The polynomial order to use.
print ('Num of training samples: ',num_data)
print('Num of testing samples: ',num_pred_data)

Now let's build the *basis* matrices $\Phi$ to represent the training data, where each column is raising the input year $X$ to various powers.

In [ ]:
Phi = np.zeros((num_data, order+1))
Phi_pred = np.zeros((num_pred_data, order+1))
for i in range(0, order+1):
    Phi[:, i:i+1] =  #.....write your code here 
    Phi_pred[:, i:i+1] = #.....write your code here 

In [ ]:
Phi_pred

### Fitting the model

Now we can solve for the regression weights and make predictions both for the training data points, and the test data points. That involves solving the linear system given by

$$\Phi' \Phi \mathbf{w} = \Phi' \mathbf{y}$$

with respect to $\mathbf{w}$.

In [ ]:
# solve the linear system
w =  #.....write your code here 
print(w)

In [ ]:
#use resulting vector to make predictions at the training points and test points
f =  #.....write your code here 
f_pred = #.....write your code here 

In [ ]:
f_pred.shape

In [ ]:
# compute the residual sum of squares (error)
RSS =   (((y-f)**2).sum())  #.....write your code here 
RSS

In [ ]:
#Now we have the fit and the error, so let's plot the fit and the error.

print("The error is: %2.4f"%RSS)
plt.plot(x_pred, f_pred)
plt.plot(x, y, 'rx')
ax = plt.gca()
ax.set_title('Predictions for Order 4')
ax.set_xlabel('PM 2.5')
ax.set_ylabel('AQI')

Now use the loop structure below to compute the error for different model orders.

In [ ]:
# import the time model to allow python to pause.
# import the IPython display module to clear the output.
import time
from IPython.display import clear_output

error_list = []
max_order = 6
fig1=plt.figure(figsize=(15,2*max_order))
index=1

for order in range(0, max_order+1):
    # 1. build the basis set
    Phi = np.zeros((num_data, order+1))
    Phi_pred = np.zeros((num_pred_data, order+1))
    for i in range(0, order+1):
        Phi[:, i:i+1] = x**i
        Phi_pred[:, i:i+1] = x_pred**i
    # 2. solve the linear system
    w = np.linalg.solve(np.dot(Phi.T, Phi), np.dot(Phi.T, y))

    # 3. make predictions at training and test points
    f = np.dot(Phi, w)
    f_pred = np.dot(Phi_pred, w)
    
    # 4. compute the training error and append it to a list.
    RSS = (((y-f)**2).sum())  
    error_list.append(RSS)
    
    # 5. plot the predictions
    fig1.add_subplot(max_order+1,2,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim((2.5, 500))
    if (order <7):
        plt.title('Predictions for Order ' + str(order) + ' model.')
    
    
    fig1.add_subplot(max_order+1,2,index+1)
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.6)
    plt.plot(np.arange(0, order+1), np.asarray(error_list))
    plt.xlim((0, order+1))
    plt.ylim((0, np.max(error_list)))
    if (order ==0):
        plt.title('Training Error')
    index= index+2

plt.show()
#display(fig)
print('Training error list: ',np.around(error_list,2))

**Home Task:** Looks like a great fit. Does that mean we can stop here, our job is done? You might want to try an order 20 or higher model, also to see if the fits continue to improve with higher order models.

**Discussion:** What do you think might happen if we try to fit an order 100 model to this data? Is this even a reasonable thing to try?

### Model Generalization using Hold-out Validation
The error we computed above is the training error. It doesn't assess the model's generalization ability, it only assesses how well it's performing on the given training data. 


In hold out validation, we keep back some of the training data for assessing generalization performance. To perform hold out validation, we first remove the hold out set from training data to create train, test (or prediction) or validation set (hold out). 

In [ ]:
import random
# Create a training set
x_train = x
y_train = y

indices_hold_out=random.sample(range(0,84), 30)
#print(indices_hold_out)
x_train = np.delete(x, indices_hold_out)[:,None]
y_train = np.delete(y, indices_hold_out)[:,None]

# Create a hold out set
x_hold_out = x[indices_hold_out]
y_hold_out = y[indices_hold_out]


print ('Whole dataset size', x.shape)
print('Train split size: ', x_train.shape)
print('Hold-Out split size: ', x_hold_out.shape)

# Now use the training set and hold out set.

Now you have the training and hold out data, you should be able to use the code above to evaluate the model on the hold out data. Do this in the code block below.



In [ ]:
from IPython.display import clear_output

error_list = []
max_order = 6
fig2=plt.figure(figsize=(12,max_order*2))
index = 1
for order in range(0, max_order+1):
    # 1. build the basis set using x_train, x_hold_out, and prediction set
    Phi = np.zeros((x_train.shape[0], order+1))
    Phi_pred = np.zeros((num_pred_data, order+1))
    Phi_hold_out = np.zeros((x_hold_out.shape[0], order+1))
    for i in range(0, order+1):
        Phi[:, i:i+1] = x_train**i
        Phi_hold_out[:, i:i+1] = x_hold_out**i
        Phi_pred[:, i:i+1] = x_pred**i
        
    # 2. solve the linear system
    w = np.linalg.solve(np.dot(Phi.T, Phi), np.dot(Phi.T, y_train))

    # 3. make predictions at training, hold_out, and prediction points
    f = np.dot(Phi, w)
    f_hold_out = np.dot(Phi_hold_out, w)
    f_pred = np.dot(Phi_pred, w)
    
    # 4. compute the hold out error and append it to a list.
    error = (((y_hold_out-f_hold_out)**2).sum())   
    error_list.append(error)
    
    # 5. plot the predictions
    fig2.add_subplot(max_order+1,2,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim((2.5, 500))
    if (order <7):
        plt.title('Predictions for Order ' + str(order) + ' model.')
    
    fig2.add_subplot(max_order+1,2,index+1)
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.6)
    fig2.add_subplot(max_order+1,2,index+1)
    plt.plot(np.arange(0, order+1), np.asarray(error_list))
    plt.xlim((0, order+1))
    plt.ylim((0, np.max(error_list)))
    if (order ==0):
        plt.title('Hold out Error')
    index= index+2

plt.show()
#display(fig)
print('Holdout error list: ', np.around(error_list,2))

**Discussion:** What is going on here? Does this match your earlier findings, or your intuition about which model order was most appropriate? Why isn't hold-out error behaving the same as training error?

## Regularizing the Model, Using Ridge Regression

A nice way to limit model complexity is *regularisation* where model parameters are penalised from moving to high magnitude values (which means the model is getting overly confident).

For this exercise, we'll use a 6th order model, which you might consider much too powerful for this simple problem. As a first step, we'll preprocess the features to ensure they are all operating in a similar range, which means the weights for the 6th order features will take on radically different values to the 1st order features. (Recall that when we regularize, we encourage weights to be small. A weight associated with the higher orders of feature can be smaller than a weight associated with the other (low orders) feature and express the same thing. So, we're not letting the data decide which features to weight more heavily, we've accidentally biased it towards favoring the first to the second.)


To correct for this, and allow regularisation with a single constant, we'll normalize (z-score) the columns of training Phi to have zero mean and unit standard deviation. This same transformation is also applied to the testing basis matrices.

In [ ]:
order = 6
Phi = np.zeros((x_train.shape[0], order+1))
Phi_pred = np.zeros((num_pred_data, order+1))
Phi_hold_out = np.zeros((x_hold_out.shape[0], order+1))
for i in range(0, order+1):
    Phi[:, i:i+1] = x_train**i
    if i > 0:
        mean = Phi[:, i:i+1].mean()
        std = Phi[:, i:i+1].std()
        print(i,mean,std)
    else: # as the first column is constant, need to avoid divide by zero 
        mean = 0
        std = 1
    
    Phi[:, i:i+1] = (Phi[:, i:i+1] - mean) / std
    Phi_hold_out[:, i:i+1] = (x_hold_out**i - mean) / std
    Phi_pred[:, i:i+1] = (x_pred**i - mean) / std


In [ ]:
#Next we'll perform training, trying out different values of the regularisation coefficient, lambda.

error_list = []
train_error_list = []
lambdas = [1e-10, 1e-6, 1e-4, 1e-2, 1, 10] 
order = 6
#fig, axes = plt.subplots(nrows=1, ncols=3)
fig3=plt.figure(figsize=(16,order*4))
index =1
for l, lamba in enumerate(lambdas):
    # 1. build the basis set using x_train, x_hold_out
    # done above
        
    # 2. solve the linear system
    w =  np.linalg.solve(np.dot(Phi.T, Phi) + lamba * np.eye(order+1), np.dot(Phi.T, y_train))#.....write your code here 

    # 3. make predictions at training and test points
    f = np.dot(Phi, w)
    f_hold_out = np.dot(Phi_hold_out, w)
    f_pred = np.dot(Phi_pred, w)
    
    # 4. compute the hold and training error and append it to a list.
    error = (((y_hold_out-f_hold_out)**2).sum())  
    error_list.append(error)
    train_error = (((y_train-f)**2).sum())   
    train_error_list.append(train_error)
    
    # 5. plot the predictions
    fig3.add_subplot(len(lambdas)+1,3,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim(2.5, 500)
    if (l==0):
        plt.title('Pred. for Lambda ' + str(lamba))
    else: 
        plt.title(str(lamba))
        
    fig3.add_subplot(len(lambdas)+1,3,index+1)
    plt.plot(lambdas[:l+1], np.asarray(error_list))
    plt.xlim((min(lambdas), max(lambdas)))
    plt.xscale('log')
    plt.ylim(10000, 70000)
    if (l==0):
        plt.title('Held-out Error (validation/testing)')
    
    
    fig3.add_subplot(len(lambdas)+1,3,index+2)
    plt.plot(lambdas[:l+1], np.asarray(train_error_list))
    plt.xlim(min(lambdas), max(lambdas))
    plt.xscale('log')
    plt.ylim(10000, 50000)
    if (l == 0):
        plt.title('Training Error')
    index= index+3

plt.show()
#display(fig)
print('Holdout error list: ',np.around(error_list,2))

**Discussion:** What setting gives the best heldout performance? How does this relate to the training error, and can you describe whether you see evidence of overfitting or underfitting?

Now that you have a good understanding of what's going on under the hood in Linear Regression, you can use the functionality in `sklearn` to solve linear regression problems you encounter in the future. Using the `LinearRegression` module, fitting a linear regression model becomes a one-liner as shown below.

In [ ]:
# Try at home
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x, y)
lr.intercept_
lr.coef_

#### Polynomial basis functions
Since the relationship between $y$ and $x$ is non-linear, we'll apply polynomial basis expansion to degree $d$.
Specifically, we replace the original data matrix $\mathbf{X}$ by the transformed matrix $\mathbf{\Phi}$, as we have seen before in this workshop. Note that we have to include a column of ones to account for the bias term.

The function below is a wrapper around `sklearn.preprocessing.PolynomialFeatures`, which implements the above transformation on a train/test set.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def polynomial_features(X_train, X_test, degree, include_bias=True):
    """
    Augments data matrices X_train and X_test with polynomial features
    """
    poly = PolynomialFeatures(degree=degree, include_bias=include_bias)
    
    Phi_train = poly.fit_transform(X_train)
    Phi_test = poly.fit_transform(X_test)
    
    return Phi_train, Phi_test
    
Phi, Phi_test = polynomial_features(x_train, x_test, 9)

In [ ]:
Phi_test

### References
1. Pattern Recognition and Machine Learning, Christopher Bishop, New York, Springer,  2006. (Chapter 3)
2. Machine learning: A Probabilistic Perspective, Kevin Murphy, MIT Press, 2012. (Chapters 17, 18)
3. Statistical Machine Learning Course Workshop, 2015, University of Melbourne, Australia